**Constitution des jeux de données pour le Challenge de la Data party**

*Les 30 ans de la MTD*

# Les jeux de données TETIS publiés

Champs d'intérêt:

- Localisation de la production : latestVersion.metadataBlocks.geospatial.fields

In [3]:
import configparser
import requests
import pandas as pd
import numpy as np

In [4]:
credential_file = "./../credentials.ini"
credential_config = configparser.ConfigParser()
credential_config.read(credential_file)
print(credential_config.sections())

['DATA_INRAE', 'PREPROD_DATA_INRAE', 'CIRAD']


In [5]:
# Paramétrage API data.inrae.fr
API_TOKEN_data_inrae = credential_config['DATA_INRAE']['API_TOKEN']
url_rdg = "https://entrepot.recherche.data.gouv.fr/"
headers_data_inrae = {'Accept': 'application/json',
                     'X-Dataverse-key': API_TOKEN_data_inrae
}

# Paramétrage API https://dataverse.cirad.fr/
API_TOKEN_data_cirad = credential_config['CIRAD']['API_TOKEN']
url_cirad = "https://dataverse.cirad.fr/"
headers_data_cirad = {'Accept': 'application/json',
                     'X-Dataverse-key': API_TOKEN_data_cirad
}

In [15]:
# get the total size of a dataverse
def storage_szie_of_dataverse(dataverse, url=url_rdg, header=headers_data_inrae):
    url_dataverse = url + "api/dataverses/" + dataverse
    reponse = requests.get(url_dataverse + "/storagesize", headers=header).json()["data"]
    return reponse

# get dataset ID from a dataverse
def datasetID_from_dataverse(dataverse, url=url_rdg, header=headers_data_inrae):
    url_dataverse = url + "/api/dataverses/" + dataverse
    try:
        reponse = requests.get(url_dataverse + "/contents", headers=header).json()["data"]
    except:
        print(requests.get(url_dataverse + "/contents", headers=header).json())
        reponse = {}
    return reponse

# get all data from a dataverse
def datasets_info_from_datasetID(dataverse_content, url=url_rdg, header=headers_data_inrae):
    """
    """
    list_datasets = []
    list_dataverses = []
    for dataset in dataverse_content:
        try: 
            dataset_info = requests.get(url + "/api/datasets/" + str(dataset["id"]), headers=header).json()["data"]
            list_datasets.append(dataset_info)
        except: # it's not a dataset but a dataverse
            list_dataverses.append(str(dataset["title"]).replace(" ", "_"))
    return list_datasets, list_dataverses

# get view (total and uniques), download (unique & totla) and citation of a dataset
def get_metrics_of_datasets(doi):
    api_metrics = ["viewsTotal", "viewsUnique", "downloadsTotal", "downloadsUnique", "citations"]
    dic_api_metrics = {}
    for metric in api_metrics:
        url = url_rdg + "/api/datasets/:persistentId/makeDataCount/" + metric + "?persistentId=" + doi
        try: 
            reponse = requests.get(url, headers=headers_data_inrae).json()["data"][metric]
        except:
            reponse = 0
        dic_api_metrics[metric] = reponse
    print(dic_api_metrics)
    return dic_api_metrics

def get_metrics_of_datasets_cirad(doi):
    api_metrics = ["viewsTotal", "viewsUnique", "downloadsTotal", "downloadsUnique", "citations"]
    dic_api_metrics = {}
    for metric in api_metrics:
        url = url_cirad + "/api/datasets/:persistentId/makeDataCount/" + metric + "?persistentId=" + doi
        try: 
            reponse = requests.get(url, headers=headers_data_cirad).json()["data"][metric]
        except:
            reponse = 0
        dic_api_metrics[metric] = reponse
    return dic_api_metrics

def extract_country_city(fields):
    # Check if fields is None or NaN, and return None for both columns if true
    if not isinstance(fields, list):
        return pd.Series([None, None])

    country = None
    city = None

    # Loop through each field in the JSON structure
    for field in fields:
        if field['typeName'] == 'geographicCoverage':
            for item in field['value']:
                if 'country' in item:
                    country = item['country']['value']
                if 'city' in item:
                    city = item['city']['value']
    return pd.Series([country, city])

In [7]:
list_of_dataverse_to_explore = []
list_of_dataverse_to_explore.append("tetis")
list_datasets = []
# Jump into dataverses inside the dataverse of TETIS!
while list_of_dataverse_to_explore:
    for dataverse in list_of_dataverse_to_explore:
        print("RDG: " + dataverse)
        datasetIDs = datasetID_from_dataverse(dataverse)
        print("\t Number of objects: " + str(len(datasetIDs)))
        try:
            list_datasets_local, list_dataverses_local = datasets_info_from_datasetID(datasetIDs)
            list_datasets.extend(list_datasets_local)
            list_of_dataverse_to_explore.extend(list_dataverses_local)
            print("\t Number of datasets: " + str(len(list_datasets_local)))
            print("\t Number of dataverse: " + str(len(list_dataverses_local)))
        except:
            pass

        print("Dataverse CIRAD: " + dataverse)
        datasetIDs = datasetID_from_dataverse(dataverse, url=url_cirad, header=headers_data_cirad)
        print("\t Number of objects: " + str(len(datasetIDs)))
        try:
            list_datasets_local, list_dataverses_local = datasets_info_from_datasetID(datasetIDs, url=url_cirad, header=headers_data_cirad)
            list_datasets.extend(list_datasets_local)
            list_of_dataverse_to_explore.extend(list_dataverses_local)
            print("\t Number of datasets: " + str(len(list_datasets_local)))
            print("\t Number of dataverse: " + str(len(list_dataverses_local)))
        except:
            pass

        list_of_dataverse_to_explore.remove(dataverse)
df_datasets = pd.json_normalize(list_datasets, max_level=10)

RDG: tetis
	 Number of objects: 47
	 Number of datasets: 37
	 Number of dataverse: 10
Dataverse CIRAD: tetis
	 Number of objects: 129
	 Number of datasets: 129
	 Number of dataverse: 0
RDG: Ciron
{'status': 'ERROR', 'message': 'User @rdecoupes is not permitted to perform requested action.'}
	 Number of objects: 0
	 Number of datasets: 0
	 Number of dataverse: 0
Dataverse CIRAD: Ciron
{'status': 'ERROR', 'message': "Can't find dataverse with identifier='Ciron'"}
	 Number of objects: 0
	 Number of datasets: 0
	 Number of dataverse: 0
RDG: Projet_FederActeur
	 Number of objects: 1
	 Number of datasets: 1
	 Number of dataverse: 0
Dataverse CIRAD: Projet_FederActeur
{'status': 'ERROR', 'message': "Can't find dataverse with identifier='Projet_FederActeur'"}
	 Number of objects: 0
	 Number of datasets: 0
	 Number of dataverse: 0
RDG: GloUrb_project:_Floodplain_urbanization_at_global_scale
{'status': 'ERROR', 'message': "Can't find dataverse with identifier='GloUrb_project:_Floodplain_urbaniza

À partir du champs contenant les informations de lieu `latestVersion.metadataBlocks.geospatial.fields` (sous forme de Json), nous faisons l'extraction des Pays et Ville

In [16]:
df_datasets[['country', 'city']] = df_datasets['latestVersion.metadataBlocks.geospatial.fields'].apply(extract_country_city)

In [17]:
df_datasets

,id,identifier,persistentUrl,protocol,authority,publisher,publicationDate,storageIdentifier,latestVersion.id,latestVersion.datasetId,...,latestVersion.UNF,latestVersion.termsOfAccess,latestVersion.distributionDate,latestVersion.termsOfUse,latestVersion.metadataBlocks.cirad.displayName,latestVersion.metadataBlocks.cirad.name,latestVersion.metadataBlocks.cirad.fields,latestVersion.citationRequirements,country,city
0,121258,MVHCLQ,https://doi.org/10.15454/MVHCLQ,doi,10.15454,Recherche Data Gouv,2021-10-20,inrae://10.15454/MVHCLQ,265423.0,121258.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,121785,4TEO6H,https://doi.org/10.15454/4TEO6H,doi,10.15454,Recherche Data Gouv,2021-10-15,inrae://10.15454/4TEO6H,263497.0,121785.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,121900,GR5EFS,https://doi.org/10.15454/GR5EFS,doi,10.15454,Recherche Data Gouv,2021-10-20,inrae://10.15454/GR5EFS,267413.0,121900.0,...,UNF:6:1X3SlDNDFBeyWne9G6w5Nw==,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,121924,QCKBKF,https://doi.org/10.15454/QCKBKF,doi,10.15454,Recherche Data Gouv,2021-10-21,inrae://10.15454/QCKBKF,263516.0,121924.0,...,NaN,Veuillez contacter Samuel Alleaume (samuel.all...,NaN,NaN,NaN,NaN,NaN,NaN,France,San Giuliano
4,128143,T6OBBY,https://doi.org/10.15454/T6OBBY,doi,10.15454,Recherche Data Gouv,2022-04-01,inrae://10.15454/T6OBBY,264489.0,128143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,187798,F6TBHY,https://doi.org/10.57745/F6TBHY,doi,10.57745,Recherche Data Gouv,2024-02-02,inrae://10.57745/F6TBHY,270802.0,187798.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
168,187803,IG5UL8,https://doi.org/10.57745/IG5UL8,doi,10.57745,Recherche Data Gouv,2024-02-02,inrae://10.57745/IG5UL8,270803.0,187803.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
169,142633,OBEXYH,https://doi.org/10.57745/OBEXYH,doi,10.57745,Recherche Data Gouv,2022-05-24,inrae://10.57745/OBEXYH,271930.0,142633.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
170,142655,ROWLJ7,https://doi.org/10.57745/ROWLJ7,doi,10.57745,Recherche Data Gouv,2022-05-24,inrae://10.57745/ROWLJ7,264674.0,142655.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [19]:
df_datasets.to_csv(f"./../data/data_tetis.csv")